# Custom video preprocessing for WASB tennis inferenceThis notebook converts sample videos placed under `myInput/game*/sample*.mp4` into the frame/CSV layout expected by the tennis dataloader, so you can run inference with visualization overlays on CPU.

## Prerequisites- Place your MP4 files under `myInput/game*/sample*.mp4` (e.g., `myInput/game1/sample1.mp4`).- The notebook writes frames and placeholder annotations to `myInput_prepared/<game>/<sample>/`.- The generated `Label.csv` marks every frame as `visibility=0` with empty coordinates so inference runs without requiring ground-truth labels.

In [ ]:
from pathlib import Pathimport cv2import numpy as npimport pandas as pdfrom typing import Optional# Input and output rootsinput_root = Path('myInput')output_root = Path('myInput_prepared')output_root.mkdir(parents=True, exist_ok=True)# Video glob pattern (game*/sample*.mp4)video_paths = sorted(input_root.glob('game*/sample*.mp4'))print(f'Found {len(video_paths)} videos')for p in video_paths:    print(' -', p)

In [ ]:
def extract_video_to_clip(video_path: Path, clip_root: Path, ext: str = '.jpg', fps_limit: Optional[int] = None):    """    Extract frames from a video into `clip_root`, and emit a placeholder Label.csv.    Args:        video_path: path to the input MP4 file.        clip_root: directory where frames and Label.csv will be stored.        ext: image extension to use for saved frames.        fps_limit: optional integer to cap the number of frames per second (skip frames evenly).    """    clip_root.mkdir(parents=True, exist_ok=True)    cap = cv2.VideoCapture(str(video_path))    if not cap.isOpened():        raise RuntimeError(f'Cannot open video: {video_path}')    original_fps = cap.get(cv2.CAP_PROP_FPS) or 0    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    print(f'Processing {video_path} ({frame_count} frames @ {original_fps:.1f} fps)')    # Decide sampling stride if we need to limit FPS    stride = 1    if fps_limit and original_fps > fps_limit:        stride = max(1, int(round(original_fps / fps_limit)))        print(f' - downsampling frames with stride={stride}')    saved_files = []    idx = 0    saved_idx = 0    while True:        ret, frame = cap.read()        if not ret:            break        idx += 1        if idx % stride:            continue        fname = f"{saved_idx:06d}{ext}"        fpath = clip_root / fname        cv2.imwrite(str(fpath), frame)        saved_files.append(fname)        saved_idx += 1    cap.release()    print(f' - saved {len(saved_files)} frames to {clip_root}')    # Create placeholder Label.csv expected by tennis loader    df = pd.DataFrame({        'file name': saved_files,        'visibility': [0] * len(saved_files),  # 0 => not visible (no GT)        'x-coordinate': [np.nan] * len(saved_files),        'y-coordinate': [np.nan] * len(saved_files),    })    csv_path = clip_root / 'Label.csv'    df.to_csv(csv_path, index=False)    print(f' - wrote {csv_path}')    return saved_files

In [ ]:
# Run extraction for every detected videofor video_path in video_paths:    game_dir = video_path.parent.parent.name  # e.g., game1    clip_dir = video_path.stem                # e.g., sample1    clip_root = output_root / game_dir / clip_dir    extract_video_to_clip(video_path, clip_root)

## How to run inference with overlays (CPU)After preparing the clips, run the WASB tennis model on CPU and save overlay frames:```bashpython3 main.py --config-name=eval_cpu   dataset=tennis model=wasb   dataset.root_dir=$(pwd)/myInput_prepared   dataset.test.matches=[game1,game2,game3,game4]   runner.device=cpu runner.gpus=[]   runner.vis_result=true   detector.model_path=../pretrained_weights/wasb_tennis_best.pth.tar   dataloader.num_workers=2```- Overlays are saved under `outputs/eval_cpu/<timestamp>/` per clip.- Add `runner.vis_hm=true` or `runner.vis_traj=true` if you also want heatmaps or trajectory plots.```